Importing libraries

In [20]:
import json as js
from collections import defaultdict
import nltk
from tqdm import tqdm
import numpy as np
from gensim.models import Word2Vec
import tensorflow as tf
from keras.models import Model #type: ignore
from keras.layers import Input, LSTM, Dense, TimeDistributed #type: ignore

loading paths

In [3]:
with open('Config_RNN.json','r') as file:
    paths = js.load(file)

Loading initial train data

In [10]:
with open(paths["Caption_Train"],'r') as file:
    train_data = js.load(file)

Extracting imp data

In [11]:
newdict = {}
newdict["img_data"] = train_data["images"]
newdict["annotations_data"] = train_data["annotations"]

Creating Corpus

In [12]:
img_data = newdict["img_data"]
annotations_data = newdict["annotations_data"]
captions_dict = defaultdict(list)

for ann in annotations_data:
    captions_dict[ann["image_id"]].append(ann["caption"])

image_caption_data = {}
for img in img_data:
    file_id = img["file_name"]
    img_id = img["id"]
    image_caption_data[file_id] = captions_dict[img_id]

with open(paths["Corpus"], "w") as f:
    js.dump(image_caption_data, f, indent=4)


loading corpus

In [7]:
with open(paths["Corpus"], "r") as f:
    Corpus_data = js.load(f)

Loading nltk dependency

In [12]:
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/utkarsh/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/utkarsh/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

Text Tokenization

In [11]:
pos_tagged_captions = {}
noun_captions = {}

for item in tqdm(Corpus_data):
    tagged = []
    for caption in Corpus_data[item]:
        tokens = nltk.word_tokenize(caption)
        pos_tags = nltk.pos_tag(tokens)
        tagged.append(pos_tags)
    pos_tagged_captions[item] = tagged

for image_id, tagged_captions in tqdm(pos_tagged_captions.items()):
    noun_lists = []
    for tagged in tagged_captions:
        nouns = [word for word, tag in tagged if tag in ('NN', 'NNS', 'NNP', 'NNPS')]
        noun_lists.append(nouns)
    noun_captions[image_id] = noun_lists

100%|██████████| 118287/118287 [00:02<00:00, 43243.21it/s]


storing noune tags

In [ ]:
with open(paths['Nouned_Corpus'],'w') as f:
    js.dump(noun_captions,f)

In [ ]:
with open(paths["Corpus"], 'r') as f1:
    captions_data = js.load(f1)

with open(paths['Nouned_Corpus'], 'r') as f2:
    features_data = js.load(f2)

dataset = []

for image_id,nouns in tqdm(features_data.items()): 
    noun_caption = zip(nouns, captions_data[image_id])
    for noun, caption in noun_caption:
        dataset.append({
            "input": " ".join(noun),
            "output": caption
        })

with open(paths["Preprocessed_data"], "w") as out_file:
    js.dump(dataset, out_file, indent=2)

100%|██████████| 118287/118287 [00:00<00:00, 379465.09it/s]


Padding

In [ ]:
with open(paths["Preprocessed_data"], 'r') as f:
    data = js.load(f)

max_length = max(len(item['input']) for item in data)

for item in data:
    item['input'] = item['input'].ljust(max_length)

with open(paths["Padded_preprocessed_data"], 'w') as f:
    js.dump(data, f, indent=4)

print("All captions padded to length:", max_length)

All captions padded to length: 158


Loading padded captions

In [6]:
with open(paths["Padded_preprocessed_data"],'r') as f:
    padded_captions = js.load(f)

Build vocabulary and one hot encoding

In [13]:
vocab = set(word for cap in padded_captions for word in cap)
word2idx = {word: idx for idx, word in enumerate(sorted(vocab))}
idx2word = {idx: word for word, idx in word2idx.items()}
vocab_size = len(word2idx)
one_hot_captions = []

for cap in padded_captions:
    encoded = []

    for word in cap:
        one_hot = [0] * vocab_size
        one_hot[word2idx[word]] = 1
        encoded.append(one_hot)

    one_hot_captions.append(encoded)
    
one_hot_captions = np.array(one_hot_captions)

Embeddings

In [17]:
model = Word2Vec(sentences=padded_captions, vector_size=100, window=5, min_count=1, workers=4)
w2v_captions = []

for cap in padded_captions:
    encoded = [model.wv[word] for word in cap]
    w2v_captions.append(encoded)

Model

In [21]:
input_seq_len = 158
output_seq_len = 30
vector_dim = 158
hidden_units = 256 
encoder_inputs = Input(shape=(input_seq_len, vector_dim))
encoder = LSTM(hidden_units, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
decoder_inputs = tf.keras.layers.RepeatVector(output_seq_len)(state_h)
decoder_lstm = LSTM(hidden_units, return_sequences=True)
decoder_outputs = decoder_lstm(decoder_inputs, initial_state=[state_h, state_c])
decoder_dense = TimeDistributed(Dense(vector_dim))
final_outputs = decoder_dense(decoder_outputs)
model = Model(inputs=encoder_inputs, outputs=final_outputs)
model.compile(optimizer='adam', loss='mse') 
model.summary()

I0000 00:00:1745147823.971995    6108 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1745147824.395552    6108 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1745147824.400292    6108 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1745147824.407408    6108 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 158, 158)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 158,      │    424,960 │ input_layer[0][0] │
│                     │ 256), (None,      │            │                   │
│                     │ 256), (None,      │            │                   │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector       │ (None, 30, 256)   │          0 │ lstm[0][1]        │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 30, 256)   │    525,312 │ repeat_vector[0]… │
│                     │                   │            │ lstm[0][1],       │
│                     │                   │            │ lstm[0][2]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, 30, 158)   │     40,606 │ lstm_1[0][0]      │
│ (TimeDistributed)   │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 990,878 (3.78 MB)

 Trainable params: 990,878 (3.78 MB)

 Non-trainable params: 0 (0.00 B)